In [1]:
import csv


csv_file_path = 'D:\\Coding\\My_codes\\NLP\\CSE 412\\output5.csv'

data_list = []

with open(csv_file_path, 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.DictReader(csv_file)
    
    for row in csv_reader:
        code_snippet = row['code']
        summary = row['summary']
        
        data_list.append({"code": code_snippet, "summary": summary})

print(type(data_list))

dataset=data_list

source_code = [example["code"] for example in dataset]
summaries = [example["summary"] for example in dataset]



<class 'list'>


In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



tokenizer_code = Tokenizer(oov_token="<OOV>")
tokenizer_code.fit_on_texts(source_code)
sequences_code = tokenizer_code.texts_to_sequences(source_code)

tokenizer_summary = Tokenizer(oov_token="<OOV>")
tokenizer_summary.fit_on_texts(summaries)
sequences_summary = tokenizer_summary.texts_to_sequences(summaries)


padded_code = pad_sequences(sequences_code)
padded_summary = pad_sequences(sequences_summary)


model = keras.Sequential([
    keras.layers.Embedding(input_dim=len(tokenizer_code.word_index) + 1, output_dim=64, input_length=padded_code.shape[1]),
    keras.layers.LSTM(100),
    keras.layers.RepeatVector(padded_summary.shape[1]),
    keras.layers.LSTM(100, return_sequences=True),
    keras.layers.TimeDistributed(keras.layers.Dense(len(tokenizer_summary.word_index) + 1, activation='softmax'))
])


model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])


model.fit(padded_code, padded_summary, epochs=1000)


new_code = ["def add(a, b): return a + b"]
new_sequence = tokenizer_code.texts_to_sequences(new_code)
new_padded = pad_sequences(new_sequence, maxlen=padded_code.shape[1])
predicted_sequence = model.predict(new_padded)
predicted_summary = [tokenizer_summary.index_word[idx] for idx in tf.argmax(predicted_sequence, axis=-1).numpy()[0] if idx != 0]
predicted_summary = " ".join(predicted_summary)
print(f"Predicted Summary: {predicted_summary}")


Epoch 1/1000
3/3 [==============================] - 9s 30ms/step - loss: 5.2769 - accuracy: 0.2390
Epoch 2/1000
3/3 [==============================] - 0s 28ms/step - loss: 5.2449 - accuracy: 0.3738
Epoch 3/1000
3/3 [==============================] - 0s 24ms/step - loss: 5.1236 - accuracy: 0.3738
Epoch 4/1000
3/3 [==============================] - 0s 23ms/step - loss: 4.6030 - accuracy: 0.3738
Epoch 5/1000
3/3 [==============================] - 0s 20ms/step - loss: 3.9799 - accuracy: 0.3738
Epoch 6/1000
3/3 [==============================] - 0s 13ms/step - loss: 3.6645 - accuracy: 0.3738
Epoch 7/1000
3/3 [==============================] - 0s 13ms/step - loss: 3.4876 - accuracy: 0.3738
Epoch 8/1000
3/3 [==============================] - 0s 13ms/step - loss: 3.3633 - accuracy: 0.3738
Epoch 9/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.2532 - accuracy: 0.3738
Epoch 10/1000
3/3 [==============================] - 0s 11ms/step - loss: 3.1803 - accuracy: 0.3738
Epoch 11/

1/1 [==============================] - 0s 29ms/step
Predicted Summary: a function to multiplies two numbers


**Evaluation**
